# AGNews Graph version

### Sources
- Mátyás Reuters Graph notebook

In [1]:
import torch
import numpy as np

from data_prep.agnews_graph import AGNewsGraph

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")

In [2]:
agnews = AGNewsGraph(device)

Prepare AGNews dataset


Using custom data configuration default
Reusing dataset ag_news (/home/mat/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)


Compute tf.idf
Compute PMI scores
Generate edges
Generate masks
Generate feature matrix
Features mtx is 0.5602689000000001 GBs in size


In [3]:
print(np.unique(agnews.data.y[agnews.data.train_mask].cpu(), return_counts=True))
print(np.unique(agnews.data.y[agnews.data.val_mask].cpu(), return_counts=True))

(array([0, 1, 2, 3]), array([265, 232, 223, 280]))
(array([0, 1, 2, 3]), array([30, 29, 15, 26]))


In [4]:
print(sum(agnews.data.val_mask * agnews.data.train_mask * agnews.data.test_mask))
print(sum(agnews.data.train_mask))
print(sum(agnews.data.val_mask))
print(sum(agnews.data.test_mask))

tensor(0, device='cuda:0')
tensor(1000, device='cuda:0')
tensor(100, device='cuda:0')
tensor(100, device='cuda:0')


In [5]:
# GraphConv, GATConv
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(len(agnews.iton), 200)
        self.conv2 = GCNConv(200, 8)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

def eval(model, data, mask):
    _, pred = model(data).max(dim=1)
    #print(pred[mask])
    #print(data.y[mask])
    correct = pred[mask].eq(data.y[mask]).sum().item()
    acc = correct / mask.sum()
    print('Accuracy: {:.4f}'.format(acc))

In [6]:
model = Net()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)#, weight_decay=5e-4)
agnews.data.to(device)

Data(edge_attr=[694850], edge_index=[2, 694850], test_mask=[11835], train_mask=[11835], val_mask=[11835], x=[11835, 11835], y=[11835])

In [7]:
from tqdm.notebook import tqdm

model.train()
for epoch in tqdm(range(40)):
    optimizer.zero_grad()
    out = model(agnews.data)
    # We might want to use the "weight" parameter for the loss with unbalanced dataset
    # since with a low learning rate the model just assigns every doc to class "earn"
    # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    loss = F.cross_entropy(out[agnews.data.train_mask], agnews.data.y[agnews.data.train_mask])
    print('Loss:', loss.item())
    loss.backward()
    optimizer.step()
    print(np.unique(out[agnews.data.train_mask].max(dim=1)[1].detach().cpu().numpy(), return_counts=True))
    #print(np.unique(r8.data.y[r8.data.train_mask].detach().cpu().numpy(), return_counts=True))
    eval(model, agnews.data, agnews.data.val_mask)

Loss: 2.0793919563293457
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([131, 187,  41, 105,   4, 199, 148, 185]))
Accuracy: 0.6000
Loss: 1.6143388748168945
(array([0, 1, 2, 3]), array([338, 177,  90, 395]))
Accuracy: 0.4700
Loss: 1.3217287063598633
(array([0, 1, 3]), array([487,   3, 510]))
Accuracy: 0.4400
Loss: 1.1469839811325073
(array([0, 1, 2, 3]), array([245, 172,  71, 512]))
Accuracy: 0.6500
Loss: 0.937404453754425
(array([0, 1, 2, 3]), array([246, 250, 247, 257]))
Accuracy: 0.7600
Loss: 0.7050908803939819
(array([0, 1, 2, 3]), array([285, 232, 181, 302]))
Accuracy: 0.7900
Loss: 0.47222384810447693
(array([0, 1, 2, 3]), array([257, 233, 209, 301]))
Accuracy: 0.7800
Loss: 0.3010525405406952
(array([0, 1, 2, 3]), array([256, 234, 236, 274]))
Accuracy: 0.8200
Loss: 0.17584934830665588
(array([0, 1, 2, 3]), array([260, 234, 218, 288]))
Accuracy: 0.8100
Loss: 0.10398869961500168
(array([0, 1, 2, 3]), array([263, 233, 217, 287]))
Accuracy: 0.8400
Loss: 0.06586113572120667
(array([0, 1, 2, 3

In [8]:
eval(model, agnews.data, agnews.data.test_mask)

Accuracy: 0.8100
